In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
#超参数
input_size = 784
hidden_size = 500
num_class = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
#MNIST 数据集
train_dataset = dsets.MNIST(root='../data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = dsets.MNIST(root='../data', train=False, transform=transforms.ToTensor(), download=True)

In [4]:
#加载数据集
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
#构建网络模型
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_class):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_class)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
net = Net(input_size, hidden_size, num_class)
net.cuda()

Net (
  (fc1): Linear (784 -> 500)
  (relu): ReLU ()
  (fc2): Linear (500 -> 10)
)

In [8]:
#定义损失函数和优化函数
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [9]:
#训练网络
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28)).cuda()
        labels = Variable(labels).cuda()
        
        #前向+后向+优化
        optimizer.zero_grad()
        outputs = net(images)
        
        #调用损失函数，计算输出值和真实值之间的差
        loss = criterion(outputs, labels)
        #反向传播
        loss.backward()
        #开始迭代优化
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/5], Step [100/600], Loss: 0.3720
Epoch [1/5], Step [200/600], Loss: 0.5130
Epoch [1/5], Step [300/600], Loss: 0.1968
Epoch [1/5], Step [400/600], Loss: 0.2184
Epoch [1/5], Step [500/600], Loss: 0.1336
Epoch [1/5], Step [600/600], Loss: 0.1931
Epoch [2/5], Step [100/600], Loss: 0.1308
Epoch [2/5], Step [200/600], Loss: 0.0768
Epoch [2/5], Step [300/600], Loss: 0.0924
Epoch [2/5], Step [400/600], Loss: 0.1660
Epoch [2/5], Step [500/600], Loss: 0.1752
Epoch [2/5], Step [600/600], Loss: 0.0228
Epoch [3/5], Step [100/600], Loss: 0.1139
Epoch [3/5], Step [200/600], Loss: 0.0626
Epoch [3/5], Step [300/600], Loss: 0.0454
Epoch [3/5], Step [400/600], Loss: 0.0363
Epoch [3/5], Step [500/600], Loss: 0.1305
Epoch [3/5], Step [600/600], Loss: 0.0313
Epoch [4/5], Step [100/600], Loss: 0.0173
Epoch [4/5], Step [200/600], Loss: 0.0877
Epoch [4/5], Step [300/600], Loss: 0.0253
Epoch [4/5], Step [400/600], Loss: 0.0325
Epoch [4/5], Step [500/600], Loss: 0.0466
Epoch [4/5], Step [600/600], Loss:

In [10]:
#测试网络
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28)).cuda()
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 98 %


In [11]:
#保存模型
torch.save(net.state_dict(), 'model.pkl')